In [1]:
import os

os.environ['XILINX_XRT'] = '/opt/xilinx/xrt'
os.environ['LD_LIBRARY_PATH'] = '/opt/xilinx/xrt/lib'
os.environ['PATH'] = '/tools/Xilinx/Vitis/2023.2/bin:/opt/xilinx/xrt/bin:/tools/conda/envs/jupyterhub-env/bin:/tools/conda/condabin:/sbin:/bin:/usr/sbin:/usr/bin'
os.environ['PYTHONPATH'] = '/opt/xilinx/xrt/python'

In [2]:
import os
import numpy as np
import time
import sys
import csv
from datetime import datetime
import statistics
import threading

In [3]:
!wget -nc bondmachine.fisica.unipg.it/firmwares/bmshell.xclbin

File ‘bmshell.xclbin’ already there; not retrieving.



In [4]:
import pynq

ol = pynq.Overlay("bmshell.xclbin")
bmkernel = ol.krnl_bondmachine_rtl_1

/home/gibianch/.local/lib/python3.8/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)
/home/gibianch/.local/lib/python3.8/site-packages/pydantic/_internal/_config.py:318: UserWarning: Valid config keys have changed in V2:
* 'underscore_attrs_are_private' has been removed
  warnings.warn(message, UserWarning)


In [5]:
BATCH_SIZE = 16
INPUT_SHAPE  = (BATCH_SIZE)
OUTPUT_SHAPE = (BATCH_SIZE)
send_data = np.zeros(INPUT_SHAPE)

In [6]:
in_buffer = pynq.allocate(INPUT_SHAPE, np.int16)
out_buffer = pynq.allocate(OUTPUT_SHAPE, np.int16)
mutex = 0

In [7]:
def write_data():
    global mutex
    while True:
        # update every seconds the output buffer
        time.sleep(1)
        if mutex == 1:
            continue
        mutex = 1
        in_buffer[:]=send_data
        in_buffer.sync_to_device()
        bmkernel.call(in_buffer, out_buffer)
        out_buffer.sync_from_device()
        mutex = 0
        #print("output received: ", out_buffer)
        
write_data_thread = threading.Thread(target=write_data)
write_data_thread.start()

In [8]:
def exec_transfer(data_to_transfer):
    in_buffer = pynq.allocate(INPUT_SHAPE, np.int16)
    in_buffer[:]=data_to_transfer
    in_buffer.sync_to_device()
    bmkernel.call(in_buffer, out_buffer)
    out_buffer.sync_from_device()

def write_user_data(user_input):
    
    input_len = len(user_input)
    new_array = [input_len]
    for c in user_input:
        new_array.append(ord(c))

    if len(new_array) < BATCH_SIZE:
        for i in range(0, 16-len(new_array)):
            new_array.append(0)
            
        exec_transfer(new_array)

        local_out_buffer = list(out_buffer)
        lenght_return = len(out_buffer)

        returned_string = ""
        for i in range(1, lenght_return):
            returned_string += chr(local_out_buffer[i])
        
        print("reply from firmware: ",returned_string)
        
    elif len(new_array) > BATCH_SIZE:
        n = len(new_array) // 16
        arrays_to_send = [new_array[i * 16:(i + 1) * 16] for i in range(n)]
        
        if len(new_array) % 16 != 0:
            remaining_array = new_array[n * 16:]
            for i in range(0, 16-len(remaining_array)):
                remaining_array.append(0)
            arrays_to_send.append(remaining_array)
        
        final_string = ""
        cnt = 0
        for array_to_send in arrays_to_send:
            exec_transfer(array_to_send)
            local_out_buffer = list(out_buffer)
            lenght_return = len(out_buffer)

            returned_string = ""
            starting_range = 1 if cnt == 0 else 0
            for i in range(starting_range, lenght_return):
                returned_string += chr(local_out_buffer[i])
            
            final_string += returned_string
            
            cnt = cnt + 1
            
        print("reply from firmware: ",final_string)

In [9]:
def ask_user_input():
    global mutex
    if mutex == 1:
        print("Wait to finish transfer")
        time.sleep(0.1)
        ask_user_input()
    else:
        mutex = 1
        user_input = input("")
        write_user_data(user_input)
        mutex = 0

while True:
    ask_user_input()

 ciao


[  0   0   0   0   0   4  99 105  97 111   0   0   0   0   0   0]
reply from firmware:      ciao      


 1234567


[ 0  0  0  0  0  0  0  0  7 49 50 51 52 53 54 55]
reply from firmware:         1234567


 prova


[  0   0   0   0   0   0   5 112 114 111 118  97   0   0   0   0]
reply from firmware:       prova    


 ciao


[  0   0   0   0   0   4  99 105  97 111   0   0   0   0   0   0]
reply from firmware:      ciao      


KeyboardInterrupt: Interrupted by user